In [1]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
import os
import random

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# Import Data

In [2]:
# Except 5, 20
# C-H 52 #494번 index ~ 545번
# C-C 208 #1116번 index ~ 1323번
OriginData1 = np.loadtxt("1_Top.txt")
OriginData2 = np.loadtxt("2_Top.txt")
OriginData3 = np.loadtxt("3_Top.txt")
OriginData4 = np.loadtxt("4_Top.txt")
OriginData6 = np.loadtxt("6_Top.txt")
OriginData7 = np.loadtxt("7_Top.txt")
OriginData8 = np.loadtxt("8_Top.txt")
OriginData9 = np.loadtxt("9_Top.txt")
OriginData10 = np.loadtxt("10_Top.txt")
OriginData11 = np.loadtxt("11_Top.txt")
OriginData12 = np.loadtxt("12_Top.txt")
OriginData13 = np.loadtxt("13_Top.txt")
OriginData14 = np.loadtxt("14_Top.txt")
OriginData15 = np.loadtxt("15_Top.txt")
OriginData16 = np.loadtxt("16_Top.txt")
OriginData17 = np.loadtxt("17_Top.txt")
OriginData18 = np.loadtxt("18_Top.txt")
OriginData19 = np.loadtxt("19_Top.txt")
OriginData21 = np.loadtxt("21_Top.txt")
OriginData22 = np.loadtxt("22_Top.txt")
OriginData23 = np.loadtxt("23_Top.txt")
OriginData24 = np.loadtxt("24_Top.txt")
OriginData25 = np.loadtxt("25_Top.txt")
OriginData26 = np.loadtxt("26_Top.txt")
OriginData27 = np.loadtxt("27_Top.txt")
OriginData = np.loadtxt("FTIR Top Data.txt")

In [3]:
np.shape(OriginData[0:100,:])
#1738

(100, 2)

In [4]:
CH = np.loadtxt("C-H.txt")
CC = np.loadtxt("C-C.txt")

In [5]:
np.shape(CH)

(52, 2)

In [6]:
np.shape(CC)

(208, 2)

In [7]:
FCH = np.loadtxt("First to CH.txt")
FCC = np.loadtxt("First to CC.txt")

In [8]:
np.shape(FCH)

(494, 2)

In [9]:
np.shape(FCC)

(1116, 2)

# Training Parameters

In [10]:
learning_rate = 0.01
num_steps = 30000
batch_size = 434


display_step = 1000
examples_to_show = 10

# Network Parameters

In [11]:
num_hidden_1 = 869 # 1st layer num features
num_hidden_2 = 434 # 2nd layer num features (the latent dim)
num_input = 1738 # MNIST data input (img shape: 28*28) 

# tf Graph Input

In [12]:
X = tf.placeholder("float", [None,num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
}
print(X)

Tensor("Placeholder:0", shape=(?, 1738), dtype=float32)


# Building the Encoder

In [13]:
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2

# Building the Decoder

In [14]:
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model

In [15]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction

In [16]:
y_pred = decoder_op

y_pred_beforeCH = y_pred[:,0:494]
y_pred_CH = y_pred[:,494:546]
y_pred_betweenCHandCC = y_pred[:,546:1116]
y_pred_CC = y_pred[:,1116:1324]
y_pred_afterCC = y_pred[:,1324:]

# Targets (labels) are the input data

In [17]:
y_true = X

y_true_beforeCH = y_true[:,0:494]
y_true_CH = y_true[:,494:546]
y_true_betweenCHandCC = y_true[:,546:1116]
y_true_CC = y_true[:,1116:1324]
y_true_afterCC = y_true[:,1324:]

# Define loss and optimizer, minimize the squared error

In [18]:
loss = tf.reduce_mean(tf.abs(y_pred_beforeCH - y_true_beforeCH)) + tf.reduce_mean(tf.abs(y_pred_CH - 10 * y_true_CH)) + tf.reduce_mean(tf.abs(y_pred_betweenCHandCC - y_true_betweenCHandCC)) + tf.reduce_mean(tf.abs(y_pred_CC - 10 * y_true_CC)) + tf.reduce_mean(tf.abs(y_pred_afterCC - y_true_afterCC))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

# Initialize the variables (i.e. assign their default value)

In [19]:
init = tf.global_variables_initializer()

# Start Training

Start a new TF session

In [23]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    # Training
    k=0
    for i in range(1, num_steps+1):
        if k % 25 == 0:
            k = 0
        # Prepare Data
        # Get the next batch of MNIST data (only images are needed, not labels)
        batch_x= OriginData[k*batch_size:(k+1)*batch_size,:]

        # Run optimization op (backprop) and cost op (to get loss value)
        l = sess.run([optimizer, loss], feed_dict={X: batch_x})
        print("batch")
        print(tf.shape(batch_x))
        print("X")
        print(tf.shape(X))
        k=k+1
        # Display logs per step
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, l))

    # Testing
    # Encode and decode images from test set and visualize their reconstruction.
    n = 4
    for i in range(n):
        # MNIST test set
        batch_x = OriginData[i*batch_size:(i+1)*batch_size,:]
        # Encode and decode the digit image
        g = sess.run(decoder_op, feed_dict={X: batch_x})
        print(batch_x = OriginData[i*batch_size:(i+1)*batch_size,:])
        print(g)

ValueError: Cannot feed value of shape (434, 2) for Tensor 'Placeholder:0', which has shape '(?, 1738)'